In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn import linear_model
from rdkit import Chem
from rdkit.Chem import AllChem

In [94]:
"""
Read in train and test as Pandas DataFrames
"""
print("here")
df_train = pd.read_csv("train_small.csv")
print("halfway")
df_test = pd.read_csv("test_small.csv")
print("done")

here
halfway
done


In [95]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98


In [96]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
#store gap values
Y_train = df_train.gap.values
# Y_test = df_test.gap.values
X_train_smiles = df_train.smiles
X_test_smiles = df_test.smiles
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
# df_test = df_test.drop(['id'], axis=1)
#delete 'gap' column
# df_train = df_train.drop(['gap'], axis=1)

In [98]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,Id,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap,smiles
0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...
1,NaN,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...
2,NaN,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...
3,NaN,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...
4,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1


In [99]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [100]:
#Drop the 'smiles' column
all_smiles = df_all.smiles
"""
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape
"""

'\ndf_all = df_all.drop([\'smiles\'], axis=1)\nvals = df_all.values\nX_train = vals[:test_idx]\nX_test = vals[test_idx:]\nprint "Train features:", X_train.shape\nprint "Train gap:", Y_train.shape\nprint "Test features:", X_test.shape\n'

In [101]:
# LR = LinearRegression()
# LR.fit(X_train, Y_train)
# LR_pred = LR.predict(X_test)

In [102]:
"""
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)
"""

'\nRF = RandomForestRegressor()\nRF.fit(X_train, Y_train)\nRF_pred = RF.predict(X_test)\n'

In [103]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [104]:
# write_to_file("sample1.csv", LR_pred)
# write_to_file("sample2.csv", RF_pred)

In [105]:
#mean_squared_error(LR_pred, Y_test)

In [106]:
#mean_squared_error(RF_pred, Y_test)

In [107]:

X_scaled = preprocessing.scale(X_train)
RR = RidgeCV(alphas = np.linspace(0.01, 10, 100))
RR.fit(X_scaled, Y_train)
RR_pred = RR.predict(X_test)
print "mean squared error", mean_squared_error(RR_pred, Y_test)
print "RR.alpha", RR.alpha_


'\nX_scaled = preprocessing.scale(X_train)\nRR = RidgeCV(alphas = np.linspace(0.01, 10, 100))\nRR.fit(X_scaled, Y_train)\nRR_pred = RR.predict(X_test)\nprint "mean squared error", mean_squared_error(RR_pred, Y_test)\nprint "RR.alpha", RR.alpha_\n'

In [109]:
# morgans = list()
morgans = map(lambda x: list(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x),2,nBits=1024)), all_smiles)
# print morgans[1]
morgans_df = pd.DataFrame(data=morgans)
print morgans_df.shape
print morgans_df.head()
morgans_vals = morgans_df.values
X_train_morgans = morgans_vals[:test_idx]
X_test_morgans = morgans_vals[test_idx:]
LR = LinearRegression()
LR.fit(X_train_morgans, Y_train)
LR_pred = LR.predict(X_test_morgans)
write_to_file("out.csv", LR_pred)
# print "mean squared error", mean_squared_error(LR_pred, Y_test)
# for i in range(df_all.shape()):
    # morgans.append(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles[i]),2,nBits=256)

(19996, 1024)
   0     1     2     3     4     5     6     7     8     9     ...   1014  \
0     0     0     0     0     0     0     0     1     0     0  ...      0   
1     0     0     0     0     0     0     0     0     0     0  ...      0   
2     0     0     0     0     0     0     0     0     0     0  ...      0   
3     0     0     0     0     0     0     0     0     0     0  ...      0   
4     0     0     0     0     0     0     0     0     0     0  ...      0   

   1015  1016  1017  1018  1019  1020  1021  1022  1023  
0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  

[5 rows x 1024 columns]
